# **AM 207**: Homework 5

Verena Kaynig-Fittkau and Pavlos Protopapas  <br>
**Due: 11.59 P.M. Thursday April 14th, 2016**

### Note: This homework is only for one week

### Instructions:

+ Upload your answers in an ipython notebook to Canvas.

+ We will provide you imports for your ipython notebook. Please do not import additional libraries.

+ Your individual submissions should use the following filenames: AM207_YOURNAME_HW5.ipynb

+ Your code should be in code cells as part of your ipython notebook. Do not use a different language (or format). 

+ **Do not just send your code. The homework solutions should be in a report style. Be sure to add comments to your code as well as markdown cells where you describe your approach and discuss your results. **

+ Please submit your notebook in an executed status, so that we can see all the results you computed. However, we will still run your code and all cells should reproduce the output when executed. 

+ If you have multiple files (e.g. you've added code files or images) create a tarball for all files in a single file and name it: AM207_YOURNAME_HW5.tar.gz or AM207_YOURNAME_HW5.zip


### Have Fun!
_ _ _ _ _

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("white")

import time
import timeit

import scipy.stats 
import pandas as pd
import pymc as pm

import re
import numpy as np

import string

/Users/charlesliu/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# Problem 1: HMM... I Think Your Text Got Corrupted!

In this problem you should use a Hidden Markov Model to correct typos in a text without using a dictionary. Your data is in two different text files:

* `Shakespeare_correct.txt` contains the words of some sonnets from Shakespeare
* `Shakespeare_typos.txt` contains the same text, but now some of the characters are corrupted

For convenience both text files only contain lower case letters a-z and spaces. 

First build a first order HMM:
* What are the hidden states and what are the observed states?
* What should you do to generate your HMM probability matrices?
* For some of the HMM parameters, you won't have enough training data to get representative probabilities.  For example, some of your probabilites might be 0. You should address this problem by adding a small pseudocount, similar to the motif finding problem from a previous assignment. 
* Implement the Viterbi algorithm and run it on a test portion that contains errors. Show that your Viterbi implementation can improve text of length 100, 500, 1000, and 2000. Note: To do this correctly you would have to withhold the part of the text that you use for testing when you estimate the parameters for you HMM. For the sake of this homework it is ok though to report training error instead of test error. Just be aware that the correction rate you are reporting most likely is a very optimistic estimate. 
* What correction rate do you get?

**Important**: Wikipedia has a nice article on [Viterbi](https://en.wikipedia.org/wiki/Viterbi_algorithm). **Please do not use the python implementation from this article!** (The lecture notebook also has the version from Wikipedia). Using dictionaries for Viterbi is really not intuitive and using numpy is typically faster. The article has very nice pseudo code that should enable you to easily program Viterbi by yourself. Please also refrain for this problem from using any other third party implementations. 

Now for a second order HMM:
By using a second order HMM, you should be able to get a better correction rate. 
* Give an intuitive explanation why a second order HMM should give better results.
* Implement your second order text correction. Hint: If you think a bit about the model you won't even have to change your Viterbi implementation. 
* Compare your correction rates against the first order model for text length of 100 and 500, (you can do 1000 as well if your computer is fast enough). 
* How well would your implementation scale to HMMs of even higher order? 

In [29]:
with open ("Shakespeare_correct.txt", "r") as txt:
    sh_correct = list(txt.readlines()[0])
with open ("Shakespeare_typos.txt", "r") as txt:
    sh_typo = list(txt.readlines()[0])

In [96]:
# dict to go from character(or space) to index number and vice versa for convenience
charToInt = dict(zip(list("abcdefghijklmnopqrstuvwxyz "), range(27)))
intToChar = dict(zip(range(27), list("abcdefghijklmnopqrstuvwxyz ")))

In [72]:
# map character sequences to number indices
sh_correct_idx = map(lambda x: charToInt[x], sh_correct)
sh_typo_idx = map(lambda x: charToInt[x], sh_typo)

In [89]:
# initialize starting, transition, and emission counts
# states and emissions are just characters or space
# initialize counts to 1 to provide pseudocount
transition = np.ones((len(charToInt), len(charToInt)))
emission = np.ones((len(charToInt), len(charToInt)))
start = np.ones((len(charToInt)))

# as stated in problem, don't need to separate out test set can just use entire corpus to train
# add a count to start for every start of a new word
# add a count to transition for every transition in correct corpus
# add an emission count for every correct -> typo corpus
start[sh_correct_idx[0]] += 1
for i in range(len(sh_correct_idx)-1):
    transition[sh_correct_idx[i],sh_correct_idx[i+1]] += 1
    emission[sh_correct_idx[i],sh_typo_idx[i]] += 1
    if sh_correct_idx[i] == charToInt[" "] and i+1 < len(sh_correct_idx):
        start[sh_correct_idx[i+1]] += 1    
emission[sh_correct_idx[len(sh_correct_idx)-1],sh_typo_idx[len(sh_correct_idx)-1]] += 1

# move from counts to probabilities
transition /= transition.sum(axis=1)[:, np.newaxis]
emission /= emission.sum(axis=1)[:, np.newaxis]
start /= np.sum(start)

# move to log probabilities
transition = np.log(transition)
emission = np.log(emission)
start = np.log(start)

In [116]:
# viterbi implementation
# obs is the text with letters mapped to charToInt
# hmm params from cell above
def viterbi(obs, start_p, trans_p, emit_p):
    # create V matrix of probabilities along path
    # dimension is len(obs) x len(start_p)
    V = np.zeros((len(obs), len(start_p)))
    V[0,:] = start_p+emit_p[:, obs[0]]
    
    # loop through sequence using V[t-1] 
    for t in range(1, len(obs)):
        for y in range(len(start_p)):
            V[t,y] = np.max(V[t-1,:] + trans_p[:, y] + emit_p[y, obs[t]])
    # decoded sequence is max index of every row, return new indices mapped back to chars
    return map(lambda x: intToChar[x], np.argmax(V, 1))

In [119]:
# print results
# first line correct
# second line typo
# third line viterbi decoding
# output number of characters corrected
# output number of new errors introduced
def comparison(num_chars):
    viterbi_res = viterbi(sh_typo_idx[0:100], start, transition, emission)
    print ''.join(sh_correct[0:num_chars]) + "(Correct)"
    print ''.join(sh_typo[0:num_chars]) + "(Typos)"
    print ''.join(viterbi_res) + "(Viterbi)"
    corrected = 0
    new_err = 0
    for i in range(num_chars):
        if sh_correct[i] == sh_typo[i] and sh_correct[i] != viterbi_res[i]:
            new_err += 1
        if sh_correct[i] != sh_typo[i] and sh_correct[i] == viterbi_res[i]:
            corrected += 1
    print "Corrected: " + str(corrected)
    print "New Errors: " + str(new_err)

In [120]:
comparison(100)

from fairest creatures we desire increase that thereby beautys rose might never die but as the riper(Correct)
fron fbirest crebtuses we eesjsf iocrease uhat therfbz bebuuys rose night never eie buu as uie siper(Typos)
fron fairest creatuses we eesise increase that thereay beattys rose night never eie but as the sioer(Viterbi)
Corrected: 13
New Errors: 3


# Extra Problem 2: Final Project Review
    
You will be contacted shortly by a TF to meet and discuss your final project proposal. Be sure to take advantage of this feedback option. Review meetings should be scheduled within the week from April 11-15. 